# Combine ESPN and Pro Football Reference Dataframes
## By: Nick Bruno

### Import libraries

In [1]:
import pandas as pd
import os
import numpy as np

### Import datasets

##### Import QBR dataset

In [7]:
qbr_data = pd.read_csv('regular_season_qbr_2006_to_2018.csv', index_col=None)

In [8]:
# Drop index column
qbr_data = qbr_data.drop(columns='Unnamed: 0')

In [9]:
# List columns
qbr_data.columns

Index(['PASS', 'PENALTY', 'PLAYER', 'PTS ADDED', 'QB PLAYS', 'RAW QBR',
       'RESULT', 'RK', 'RUN', 'TOTAL EPA', 'TOTAL QBR', 'WEEK', 'YEAR'],
      dtype='object')

In [14]:
# Checkout first 5 rows
qbr_data.head()

,PASS,PENALTY,PLAYER,PTS ADDED,QB PLAYS,RAW QBR,RESULT,RK,RUN,TOTAL EPA,TOTAL QBR,WEEK,YEAR
0,4.3,0.1,"Rex Grossman, CHI",5.5,31,92.4,W 26-0 @ GB in Wk 1,1.0,0.6,5.4,96.4,1.0,2006.0
1,8.6,-0.1,"Donovan McNabb, PHI",8.1,38,95.4,W 24-10 @ HOU in Wk 1,2.0,0.1,8.9,92.5,1.0,2006.0
2,6.5,0.2,"Chad Pennington, NYJ",6.1,43,88.1,W 23-16 @ TEN in Wk 1,3.0,-0.3,7.5,90.1,1.0,2006.0
3,8.4,-0.3,"Peyton Manning, IND",5.8,45,86.0,W 26-21 @ NYG in Wk 1,4.0,0.0,8.3,86.0,1.0,2006.0
4,1.3,0.4,"Mike Vick, ATL",1.8,30,70.5,W 20-6 @ CAR in Wk 1,5.0,1.1,3.4,79.4,1.0,2006.0


In [11]:
# Investigate 'RESULT' column
qbr_data.RESULT.iloc[:10]

0       W 26-0  @ GB in Wk 1
1     W 24-10  @ HOU in Wk 1
2     W 23-16  @ TEN in Wk 1
3     W 26-21  @ NYG in Wk 1
4      W 20-6  @ CAR in Wk 1
5    W 28-17  vs MIA in Wk 1
6       W 27-0  @ TB in Wk 1
7    W 24-17  vs DAL in Wk 1
8     W 19-16  @ WSH in Wk 1
9      L 19-17  @ NE in Wk 1
Name: RESULT, dtype: object

##### Import pro-football-reference quarterback dataset

In [12]:
pfr_data = pd.read_csv('qb_data.csv')

In [13]:
# List columns
pfr_data.columns

Index(['name', 'id', 'Year', 'Date', 'G.', 'Age', 'Tm', 'X', 'Opp', 'Result',
       'GS', 'Cmp', 'Att', 'Cmp.', 'Yds', 'TD', 'Int', 'Rate', 'Sk', 'Yds.1',
       'Y.A', 'AY.A', 'Att.1', 'Yds.2', 'Y.A.1', 'TD.1', 'TD.2', 'Pts', 'Fmb',
       'FF', 'FR', 'Yds.3', 'TD.3', 'team_win', 'start_win', 'game_count',
       'start_count'],
      dtype='object')

In [15]:
# Checkout first 5 rows
pfr_data.head()

,name,id,Year,Date,G.,Age,Tm,X,Opp,Result,...,Pts,Fmb,FF,FR,Yds.3,TD.3,team_win,start_win,game_count,start_count
0,A.J. Feeley,FeelA.00,2001,1/6/02,16,24.235,PHI,@,TAM,W 17-13,...,0,0,0,0,0,0,1,0,1,0
1,A.J. Feeley,FeelA.00,2002,12/1/02,12,25.199,PHI,NaN,STL,W 10-3,...,0,0,0,0,0,0,1,1,2,1
2,A.J. Feeley,FeelA.00,2002,12/8/02,13,25.206,PHI,@,SEA,W 27-20,...,0,0,0,0,0,0,1,1,3,2
3,A.J. Feeley,FeelA.00,2002,12/15/02,14,25.213,PHI,NaN,WAS,W 34-21,...,0,0,0,0,0,0,1,1,4,3
4,A.J. Feeley,FeelA.00,2002,12/21/02,15,25.219,PHI,@,DAL,W 27-3,...,0,0,0,0,0,0,1,1,5,4


We can tell that the pro-football-reference.com data contains more data compared to the ESPN data. This will be helpful when conducting analysis. For now, we are trying to merge the espn.com and pro-football-reference.com, so it is important to find which data is included in both dataframes and how this data can be used to correctly identify individual games, leading to an eventual natural join between the dataframes.

### Subset 'pfr_data' to only include statistics from years 2006 to 2018

The data gathered from espn.com only includes data starting from the 2006 NFL season, so we should subset the pro-football-reference.com data to only include data starting in 2006.

In [16]:
pfr_data = pfr_data[pfr_data.Year > 2005]

In [17]:
pfr_data.Year.unique() # dataframe contains data from years 2006 to 2018

array([2006, 2007, 2011, 2015, 2008, 2009, 2010, 2012, 2013, 2014, 2016,
       2017, 2018])

## Combine dataframes

Because there is no common column between the two dataframes, we will have to get creative in combining them. My initial thought is to merge them by name, year, and result. This will require some column manipulation, which I will demonstrate in the following code.

### Name column

In [20]:
# First, edit the qbr_data 'PLAYER' column to a 'name' column and a 'team' column
qbr_data['name'] = qbr_data.PLAYER.str.split(',').str[0]
qbr_data['team'] = qbr_data.PLAYER.str.split(',').str[1]

In [21]:
qbr_data[['name','team']].head()

,name,team
0,Rex Grossman,CHI
1,Donovan McNabb,PHI
2,Chad Pennington,NYJ
3,Peyton Manning,IND
4,Mike Vick,ATL


##### Compare QB names in both dataframes

In [22]:
# Get a list of each QB name from both dataframes
qbr_names = qbr_data.name.unique()
pfr_names = pfr_data.name.unique()

In [23]:
len(qbr_names)

191

In [24]:
len(pfr_names)

171

Here, we can see that espn.com dataframe contains 20 more quarterbacks compared to pro-football-reference.com. Now, its important to find a list of common quarterbacks between the dataframes, identify potential name differences in the dataframes, and understand why a quarterback may be excluded from one dataframe and not the other.

##### Identify quarterback names that occur in both dataframes

In [25]:
common_names = list(set(qbr_names).intersection(pfr_names))

In [27]:
len(common_names) # 160, 11 less than the number of QB names in the pfr dataframe

160

##### Identify QBs in pfd dataframe not in espn dataframe

In [29]:
pfr_diff = np.setdiff1d(pfr_names, common_names)
pfr_diff # most of these names are slightly different in the qbr dataset

array(['A.J. McCarron', 'Alex Tanney', 'Brett Basanez', 'Brett Favre+',
       'Craig Nall', 'Kurt Warner+', 'Michael Vick', 'Mitch Trubisky',
       'Robert Griffin', 'Terrelle Pryor', 'Tony Pike'], dtype=object)

Some of the names are slightly different in the datasets. For example, the qbr_data uses "Mike Vick" rather than "Michael Vick". I will fix these issues. Also, pfr_data adds a '+' symbol to the names of hall of fame quarterbacks. That will also be adjusted.

In [30]:
# Change hall of fame quarterbacks to exclude the '+' symbol
pfr_data['name'] = pfr_data['name'].replace('Brett Favre+', 'Brett Favre')
pfr_data['name'] = pfr_data['name'].replace('Kurt Warner+', 'Kurt Warner')

In [31]:
# Replace Names in pfr dataframe to match espn dataframe
pfr_data['name'] = pfr_data['name'].replace('Michael Vick', 'Mike Vick')
pfr_data['name'] = pfr_data['name'].replace('Robert Griffin', 'Robert Griffin III')
pfr_data['name'] = pfr_data['name'].replace('Mitch Trubisky', 'Mitchell Trubisky')
pfr_data['name'] = pfr_data['name'].replace('A.J. McCarron', 'AJ McCarron')
pfr_data['name'] = pfr_data['name'].replace('Terrelle Pryor', 'Terrelle Pryor Sr.')

Now that these changes have been made, the list of 'common names' should increase.

In [33]:
# Edit lists to account for adjustments in pfr_data
qbr_names = qbr_data.name.unique()
pfr_names = pfr_data.name.unique()
common_names_mod = list(set(qbr_names).intersection(pfr_names))
len(common_names_mod) # Now 167 common QBs

167

In [34]:
# Find the quarterbacks in the ESPN data that are not in the pro-football-reference data
np.setdiff1d(qbr_names, common_names_mod)

array(['Brian St. Pierre', 'Charlie Whitehurst', "J.T. O'Sullivan",
       'Jameis Winston', 'Jeff Driskel', 'Joe Webb III', 'Josh Johnson',
       'Josh McCown', 'Josh Rosen', 'Kellen Moore', 'Kyle Allen',
       'Lamar Jackson', 'Landry Jones', 'Matt McGloin', 'Matt Moore',
       'Nick Mullens', 'Russell Wilson', 'Rusty Smith', 'T.J. Yates',
       'Taylor Heinicke', 'Thaddeus Lewis', 'Trent Edwards', 'Troy Smith',
       'Vince Young'], dtype=object)

There are some quarterbacks that are in the ESPN QBR dataset that are not in the pro-football-reference dataset. That is likely a result in the collection of the pro-football-reference Quarterback data. Although there are some big name quarterbacks excluded from the data (Russell Wilson, Jameis Winston, Vince Young), we will continue with the analysis with the 167 common quarterbacks in both datasets.

In [35]:
# Edit dataframes to only include data on these quarterbacks
pfr_data_mod1 = pfr_data[pfr_data.name.isin(common_names_mod)]

In [36]:
qbr_data_mod1 = qbr_data[qbr_data.name.isin(common_names_mod)]

In [37]:
pfr_data_mod1.shape[0]

6449

In [38]:
qbr_data_mod1.shape[0]

6094

pro-football-reference subset contains 400 more games compared to the ESPN data. This is likely because QBR is not calculated for each quarterback every week on the ESPN website. Hopefully, however, the merged dataframe will contain close to the 6,094 rows in the qbr_data_mod1 dataframe.

### Mark home (vs) and away (@) games in both dataframes

##### Check both dataframes

In [40]:
# Check pfr dataframe
pfr_data_mod1.head(1)

,name,id,Year,Date,G.,Age,Tm,X,Opp,Result,...,Pts,Fmb,FF,FR,Yds.3,TD.3,team_win,start_win,game_count,start_count
16,A.J. Feeley,FeelA.00,2006,12/31/06,16,29.229,PHI,NaN,ATL,W 24-17,...,0,0,0,0,0,0,1,0,17,13


In [41]:
# Check espn dataframe
qbr_data_mod1.head(1)

,PASS,PENALTY,PLAYER,PTS ADDED,QB PLAYS,RAW QBR,RESULT,RK,RUN,TOTAL EPA,TOTAL QBR,WEEK,YEAR,name,team
0,4.3,0.1,"Rex Grossman, CHI",5.5,31,92.4,W 26-0 @ GB in Wk 1,1.0,0.6,5.4,96.4,1.0,2006.0,Rex Grossman,CHI


pfr_data_mod1 marks home or away in the 'X' column, while qbr_data_mod1 marks this in 'RESULT' column

In [44]:
# pfr_dat_mod1 
pfr_data_mod1.X.unique()

array([nan, '@', 'N'], dtype=object)

In [45]:
# Investigate what 'N' means
pfr_data_mod1[pfr_data_mod1.X == 'N'].iloc[:2] # these represent superbowl games ('N' means neutral)

,name,id,Year,Date,G.,Age,Tm,X,Opp,Result,...,Pts,Fmb,FF,FR,Yds.3,TD.3,team_win,start_win,game_count,start_count
275,Aaron Rodgers,RodgAa00,2010,2/6/11,20,27.066,GNB,N,PIT,W 31-25,...,0,0,0,0,0,0,1,1,156,153
1457,Ben Roethlisberger,RoetBe00,2008,2/1/09,19,26.336,PIT,N,ARI,W 27-23,...,0,0,0,0,0,0,1,1,218,216


In [46]:
pfr_data_mod1[pfr_data_mod1.X.isnull()].iloc[:2] # represents home games

,name,id,Year,Date,G.,Age,Tm,X,Opp,Result,...,Pts,Fmb,FF,FR,Yds.3,TD.3,team_win,start_win,game_count,start_count
16,A.J. Feeley,FeelA.00,2006,12/31/06,16,29.229,PHI,NaN,ATL,W 24-17,...,0,0,0,0,0,0,1,0,17,13
17,A.J. Feeley,FeelA.00,2007,11/18/07,10,30.186,PHI,NaN,MIA,W 17-7,...,0,0,0,0,0,0,1,0,18,13


In [47]:
# Replace null values with vs
pfr_data_mod1['X'] = pfr_data_mod1['X'].fillna('vs')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
# qbr_data_mod1
# Create an 'X' column like in pfr_data_mod1
qbr_data_mod1['X'] = "" # initiate new column
qbr_data_mod2 = qbr_data_mod1.assign(X=qbr_data_mod1.RESULT.str.contains('@').map({True: '@', False: 'vs'}))

# https://stackoverflow.com/questions/43290762/how-to-check-pandas-dataframe-for-true-or-false-python

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [50]:
qbr_data_mod2.head()

,PASS,PENALTY,PLAYER,PTS ADDED,QB PLAYS,RAW QBR,RESULT,RK,RUN,TOTAL EPA,TOTAL QBR,WEEK,YEAR,name,team,X
0,4.3,0.1,"Rex Grossman, CHI",5.5,31,92.4,W 26-0 @ GB in Wk 1,1.0,0.6,5.4,96.4,1.0,2006.0,Rex Grossman,CHI,@
1,8.6,-0.1,"Donovan McNabb, PHI",8.1,38,95.4,W 24-10 @ HOU in Wk 1,2.0,0.1,8.9,92.5,1.0,2006.0,Donovan McNabb,PHI,@
2,6.5,0.2,"Chad Pennington, NYJ",6.1,43,88.1,W 23-16 @ TEN in Wk 1,3.0,-0.3,7.5,90.1,1.0,2006.0,Chad Pennington,NYJ,@
3,8.4,-0.3,"Peyton Manning, IND",5.8,45,86.0,W 26-21 @ NYG in Wk 1,4.0,0.0,8.3,86.0,1.0,2006.0,Peyton Manning,IND,@
4,1.3,0.4,"Mike Vick, ATL",1.8,30,70.5,W 20-6 @ CAR in Wk 1,5.0,1.1,3.4,79.4,1.0,2006.0,Mike Vick,ATL,@


### RESULT column split to show scores and opponent

Now, we want to split up the RESULT column in qbr_data_mod2 to separately display the score and opponent in the same manner for both dataframes.

In [51]:
# First, to make it easier to follow, I will set a new dataframe, pfr_data_mod2 equal to pfr_data_mod1.
pfr_data_mod2 = pfr_data_mod1

Because we already have the 'X' column, I will go ahead and replace the 'vs' in RESULTS with '@' so that there is a consistent delimiter, making it easier to split the RESULT column

In [52]:
# Replace 'vs' with '@' in qbr_data_mod2
qbr_data_mod2.RESULT = qbr_data_mod2.RESULT.str.replace('vs','@')

In [53]:
# Split RESULTS column
qbr_data_mod2['Score'] = qbr_data_mod2.RESULT.str.split('@').str[0]
qbr_data_mod2['Opp_and_week'] = qbr_data_mod2.RESULT.str.split('@').str[1]

In [54]:
# Split 'Opp_and_week' to only show the Opponent
qbr_data_mod2['Opp'] = qbr_data_mod2.Opp_and_week.str.split('in').str[0].str.strip()

In [55]:
qbr_data_mod2.head()

,PASS,PENALTY,PLAYER,PTS ADDED,QB PLAYS,RAW QBR,RESULT,RK,RUN,TOTAL EPA,TOTAL QBR,WEEK,YEAR,name,team,X,Score,Opp_and_week,Opp
0,4.3,0.1,"Rex Grossman, CHI",5.5,31,92.4,W 26-0 @ GB in Wk 1,1.0,0.6,5.4,96.4,1.0,2006.0,Rex Grossman,CHI,@,W 26-0,GB in Wk 1,GB
1,8.6,-0.1,"Donovan McNabb, PHI",8.1,38,95.4,W 24-10 @ HOU in Wk 1,2.0,0.1,8.9,92.5,1.0,2006.0,Donovan McNabb,PHI,@,W 24-10,HOU in Wk 1,HOU
2,6.5,0.2,"Chad Pennington, NYJ",6.1,43,88.1,W 23-16 @ TEN in Wk 1,3.0,-0.3,7.5,90.1,1.0,2006.0,Chad Pennington,NYJ,@,W 23-16,TEN in Wk 1,TEN
3,8.4,-0.3,"Peyton Manning, IND",5.8,45,86.0,W 26-21 @ NYG in Wk 1,4.0,0.0,8.3,86.0,1.0,2006.0,Peyton Manning,IND,@,W 26-21,NYG in Wk 1,NYG
4,1.3,0.4,"Mike Vick, ATL",1.8,30,70.5,W 20-6 @ CAR in Wk 1,5.0,1.1,3.4,79.4,1.0,2006.0,Mike Vick,ATL,@,W 20-6,CAR in Wk 1,CAR


### Investigate Team name abbreviations

##### Change 'Opp' columns to share the same abbreviations

In [57]:
# Get unique list of team abbreviations
qbr_opp = qbr_data_mod2.Opp.unique()
pfr_opp = pfr_data_mod2.Opp.unique()

In [58]:
qbr_opp

array(['GB', 'HOU', 'TEN', 'NYG', 'CAR', 'MIA', 'TB', 'DAL', 'WSH', 'NE',
       'MIN', 'IND', 'JAX', 'BUF', 'CLE', 'PHI', 'ARI', 'SF', 'SEA', 'NO',
       'KC', 'DET', 'CHI', 'PIT', 'CIN', 'LAC', 'NYJ', 'BAL', 'ATL',
       'LAR', 'OAK', 'DEN'], dtype=object)

In [59]:
pfr_opp

array(['ATL', 'MIA', 'NWE', 'SEA', 'DAL', 'NOR', 'SFO', 'PIT', 'DEN',
       'BAL', 'SDG', 'KAN', 'HOU', 'CIN', 'STL', 'NYJ', 'MIN', 'DET',
       'TAM', 'IND', 'TEN', 'CHI', 'CAR', 'JAX', 'CLE', 'ARI', 'PHI',
       'BUF', 'WAS', 'NYG', 'OAK', 'LAR', 'GNB', 'LAC'], dtype=object)

pro-football-reference uses 3 letter abbreviations for each team. ESPN, however, uses the regular abbreviations. I will change the pro-football-reference team abbreviations to match ESPN's.

In [60]:
len(qbr_opp)

32

In [61]:
len(pfr_opp)

34

In [62]:
np.setdiff1d(qbr_opp, pfr_opp)

array(['GB', 'KC', 'NE', 'NO', 'SF', 'TB', 'WSH'], dtype=object)

In [63]:
# Replace the incorrect abbreviations in pro-football-reference data for the 'Opp' column
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('GNB', 'GB')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('KAN', 'KC')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('NWE', 'NE')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('NOR', 'NO')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('SFO', 'SF')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('TAM', 'TB')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('WAS', 'WSH')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [64]:
np.setdiff1d(pfr_opp, qbr_opp)

array(['GNB', 'KAN', 'NOR', 'NWE', 'SDG', 'SFO', 'STL', 'TAM', 'WAS'],
      dtype=object)

There are two extra abbreviations in the pro-football-reference data. This is because the San Diego Chargers and St. Louis Rams both moved to Los Angeles. This is reflected in the ESPN data, as all of the old teams were replaced with the new abbreviations. I will make the changes to the pro-football-reference data.

In [65]:
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('STL', 'LAR')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('SDG', 'LAC')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [66]:
# Make the same changes for the 'Tm' Column of pfr_data_mod2
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('GNB', 'GB')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('KAN', 'KC')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('NWE', 'NE')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('NOR', 'NO')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('SFO', 'SF')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('TAM', 'TB')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('WAS', 'WSH')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('STL', 'LAR')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('SDG', 'LAC')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

We should now have enough matching columns that we can merge the dataframes.

### Clean columns for dataframe merging

In [71]:
# List qbr columns
qbr_columns = qbr_data_mod2.columns
qbr_columns

Index(['PASS', 'PENALTY', 'PLAYER', 'PTS ADDED', 'QB PLAYS', 'RAW QBR',
       'RESULT', 'RK', 'RUN', 'TOTAL EPA', 'TOTAL QBR', 'WEEK', 'YEAR', 'name',
       'team', 'X', 'Score', 'Opp_and_week', 'Opp'],
      dtype='object')

In [72]:
# List pfr columns
pfr_columns = pfr_data_mod2.columns
pfr_columns

Index(['name', 'id', 'Year', 'Date', 'G.', 'Age', 'Tm', 'X', 'Opp', 'Result',
       'GS', 'Cmp', 'Att', 'Cmp.', 'Yds', 'TD', 'Int', 'Rate', 'Sk', 'Yds.1',
       'Y.A', 'AY.A', 'Att.1', 'Yds.2', 'Y.A.1', 'TD.1', 'TD.2', 'Pts', 'Fmb',
       'FF', 'FR', 'Yds.3', 'TD.3', 'team_win', 'start_win', 'game_count',
       'start_count'],
      dtype='object')

In [74]:
# Change pfr_data_mod2 'Result' column to 'Score' to match qbr_data_mod2
pfr_data_mod2 = pfr_data_mod2.rename(columns={'Result':'Score'})

In [75]:
# Change 'YEAR' to 'Year' in qbr_data_mod2
qbr_data_mod2 = qbr_data_mod2.rename(columns={'YEAR':'Year'})

In [76]:
# Change 'Tm' in pfr_data_mod2 to 'team'
# pfr_data_mod2 = pfr_data_mod2.rename(columns={'Tm':'Team'})
pfr_data_mod2 = pfr_data_mod2.rename(columns={'Team':'team'})

In [77]:
# Common columns
list(set(pfr_columns).intersection(qbr_columns))

['name', 'X', 'Opp']

Now we have enough similar columns to merge the dataframes.

### Merge espn.com and pro-football-reference.com dataframes

In [78]:
# Use an inner join
merged_df = pd.merge(left=qbr_data_mod2, right=pfr_data_mod2, on=['name','Year','Opp','X'], how='inner')

In [80]:
merged_df.shape # Slightly less rows of data than the desired 6094

(5979, 52)

In [81]:
merged_df.head(2)

,PASS,PENALTY,PLAYER,PTS ADDED,QB PLAYS,RAW QBR,RESULT,RK,RUN,TOTAL EPA,...,Pts,Fmb,FF,FR,Yds.3,TD.3,team_win,start_win,game_count,start_count
0,4.3,0.1,"Rex Grossman, CHI",5.5,31,92.4,W 26-0 @ GB in Wk 1,1.0,0.6,5.4,...,0,0,0,0,-4,0,1,1,8,7
1,8.6,-0.1,"Donovan McNabb, PHI",8.1,38,95.4,W 24-10 @ HOU in Wk 1,2.0,0.1,8.9,...,0,0,0,0,0,0,1,1,90,87


##### List all columns available in the merged dataframe

In [82]:
merged_df.columns

Index(['PASS', 'PENALTY', 'PLAYER', 'PTS ADDED', 'QB PLAYS', 'RAW QBR',
       'RESULT', 'RK', 'RUN', 'TOTAL EPA', 'TOTAL QBR', 'WEEK', 'Year', 'name',
       'team', 'X', 'Score_x', 'Opp_and_week', 'Opp', 'id', 'Date', 'G.',
       'Age', 'Tm', 'Score_y', 'GS', 'Cmp', 'Att', 'Cmp.', 'Yds', 'TD', 'Int',
       'Rate', 'Sk', 'Yds.1', 'Y.A', 'AY.A', 'Att.1', 'Yds.2', 'Y.A.1', 'TD.1',
       'TD.2', 'Pts', 'Fmb', 'FF', 'FR', 'Yds.3', 'TD.3', 'team_win',
       'start_win', 'game_count', 'start_count'],
      dtype='object')

In [83]:
# Take a peak at the columns we are most interested in
merged_df[['name', 'TOTAL QBR','Rate','WEEK','Year','team']].head(2)

,name,TOTAL QBR,Rate,WEEK,Year,team
0,Rex Grossman,96.4,98.6,1.0,2006.0,CHI
1,Donovan McNabb,92.5,113.3,1.0,2006.0,PHI


##### Write merged dataframe to a .csv

In [84]:
merged_df.to_csv('merged_qb_ratings_df.csv', index=False, header=True)

We now have a merged dataframe that now adds the QBR statistic from the espn.com dataset to the extensive statistics avaiable in the pro-football-reference.com dataset. This dataset contains 5,979 rows of data which represent an NFL quarterback's statistics for one game from the 2006 to 2018 NFL seasons. This merged dataframe will be used for future blog posts with the goal of "reverse engineering" the algorithm that ESPN uses to calculate quarterback rating.